* Python Multithreading

In [6]:
from threading import *
import time

def display():
    for i in range(65, 91):
        print(chr(i), end=" ")
        time.sleep(0.1)

t = Thread(target=display, name="alphabet")
t.start()

print("\nMain thread finished")

A B C D E F G H I J K L M N O P Q R S T U V W X Y Z 
Main thread finished


### Explanation of the observed behavior:

The reason you see "Main thread finished" appear before all the letters from A to Z are printed is due to the asynchronous nature of Python's `threading` module.

Here's a breakdown:

1.  **Main Thread Execution**: When `t.start()` is called, a new thread (named 'alphabet') begins executing the `display` function concurrently with the main thread. However, the **main thread does not wait** for this new thread to complete its task.
2.  **Concurrency**: The main thread immediately proceeds to the next line of code: `print("\nMain thread finished")`.
3.  **Child Thread Delay**: Meanwhile, the 'alphabet' thread is slowly printing characters, pausing for 0.1 seconds after each one (`time.sleep(0.1)`).
4.  **Race Condition**: Since the main thread doesn't wait, it often finishes its execution (printing "Main thread finished") and potentially exits before the 'alphabet' thread has had a chance to print all 26 letters, especially if the main thread's subsequent tasks are quick or if the 'alphabet' thread has a long sleep duration. In a Colab environment, sometimes the execution context for the child thread might even be terminated if the main process finishes too quickly and there are no other active cells.

To ensure all letters are printed before the "Main thread finished" message, you would typically use `t.join()` after `t.start()`. The `t.join()` method would make the main thread wait until the `t` (alphabet) thread completes its execution.

In [9]:
from threading import *
import time

def display():
    for i in range(65, 91):
        print(chr(i), end=" ")
        time.sleep(0.1)

t = Thread(target=display, name="alphabet")
t.start()
for i in range(65, 91):
  print(i, end=" ")
  time.sleep(0.1)
print("\nMain thread finished")

A 65 B 66 C 67 D 68 E 69 F 70 G 71 H 72 I 73 J 74 K 75 L 76 M 77 N 78 O 79 P 80 81Q  R 82 S 83 T 84 U 85 V 86 87 W X 88 Y 89 Z 90 
Main thread finished


Here parent and child runs simultanuosly


In [ ]:
from threading import *
import time

def display():
    for i in range(65, 91):
        print(chr(i), end=" ")
        time.sleep(0.1)

t = Thread(target=display, name="alphabet")
t.start()
t.join()# to make parent wait unti the child terminate
print("\nMain thread finished")

In [11]:
class Alphabet(Thread):
  def run(self):
    for i in range(65,91):
      print(chr(i), end=" ")
      time.sleep(0.1)

t = Alphabet()
t.start()
t.join()
print("\nMain thread finished")

A B C D E F G H I J K L M N O P Q R S T U V W X Y Z 
Main thread finished


In [18]:
def display(str):
  l.acquire()
  for x in str:
    print(x)
  l.release()
l = Lock()
t1 = Thread(target=display, args=("Hello world",))
t1.start()
t2 = Thread(target=display, args=("aya nabil",))
t2.start()
t1.join()
t2.join()

H
a


### Understanding `threading.Lock`

In concurrent programming with threads, `threading.Lock` (also known as a mutual exclusion lock or mutex) is a synchronization primitive used to protect shared resources from being accessed by multiple threads simultaneously. This prevents race conditions, where the outcome depends on the unpredictable order of execution of multiple threads.

#### How it Works:

A `Lock` object has two primary states: **locked** and **unlocked**. Initially, it's unlocked.

1.  **Acquiring the Lock**: When a thread wants to access a shared resource, it first tries to `acquire()` the lock.
    *   If the lock is `unlocked`, the thread successfully acquires it, and the lock transitions to the `locked` state. The thread then proceeds to access the shared resource.
    *   If the lock is already `locked` by another thread, the current thread will **block** (pause its execution) until the lock is `released` by the thread holding it.
2.  **Releasing the Lock**: Once a thread is finished with the shared resource, it must `release()` the lock. This makes the lock `unlocked` again, allowing another waiting thread to acquire it.

#### Key Methods of `threading.Lock`:

*   **`acquire(blocking=True, timeout=-1)`**:
    *   Attempts to acquire the lock.
    *   `blocking=True` (default): If the lock is already held, the calling thread will wait until the lock is released.
    *   `blocking=False`: If the lock is already held, the method returns `False` immediately without waiting. If successful, it returns `True`.
    *   `timeout`: Specifies a floating-point timeout period. If `timeout` is positive, it blocks for at most that many seconds and then returns `True` if the lock was acquired, or `False` otherwise. If `timeout` is `-1` (default), it waits indefinitely.
    *   Returns `True` if the lock is acquired, `False` otherwise (only relevant when `blocking=False` or `timeout` is positive).

*   **`release()`**:
    *   Releases the lock. It can only be called from the thread that currently holds the lock.
    *   Attempting to release an unlocked lock, or a lock held by another thread, will raise a `RuntimeError`.

#### How it "Knows" the Shared Resource:

The `Lock` object itself **does not directly know or manage the shared resource**. Instead, it's a convention and responsibility of the programmer to:

1.  **Identify Shared Resources**: Determine which parts of the code access data or resources that could be corrupted if multiple threads modify them concurrently (e.g., global variables, external files, print statements in certain contexts).
2.  **Associate Lock with Resource**: Create a `Lock` object specifically for that shared resource.
3.  **Encapsulate Access with Lock**: Ensure that *any* code block that accesses or modifies the shared resource is surrounded by `lock.acquire()` and `lock.release()` calls.

    ```python
    import threading
    import time

    shared_data = 0
    data_lock = threading.Lock()

    def increment_data():
        global shared_data
        for _ in range(100000):
            # Acquire the lock before accessing shared_data
            data_lock.acquire()
            try:
                shared_data += 1
            finally:
                # Release the lock after accessing shared_data
                data_lock.release()

    threads = [threading.Thread(target=increment_data) for _ in range(5)]

    for t in threads:
        t.start()
    for t in threads:
        t.join()

    print(f"Final shared_data: {shared_data}") # Expected: 500000
    ```

In this example, `data_lock` is the `Lock` object. It "knows" `shared_data` not through any internal mechanism, but because the programmer has explicitly written the code to `acquire` and `release` `data_lock` *around* the `shared_data += 1` operation. This discipline guarantees that only one thread at a time can execute the critical section that modifies `shared_data`.